These queries follow the SQLite dialect.  Some changes (such replacing the CASE with the monthname function in question
4 would be made for other dialects.

Some queries were tested using sqldf in Python 3.8.


### Question 1

In [ ]:
query1 = """SELECT *
                FROM dates
                LEFT JOIN (SELECT SUM(s.UNIT_PRICE * s.QUANTITY) as TOTAL,
                                b.BRAND_NAME,
                                DATE(s.DATE) as DATE_ONLY
                            FROM store_table s
                            INNER JOIN brand_table b
                            ON s.BRAND_ID = b.BRAND_ID
                                WHERE HOLDING_COMPANY_NAME = "Beam Suntory"
                            GROUP BY BRAND_NAME, DATE(s.DATE)) AS sub
                ON DATE(DATE) = sub.DATE_ONLY
                WHERE DATE(DATE) BETWEEN '2020-03-13' AND '2020-06-27'"""



### Question 2

In [ ]:
query2 = """SELECT s.PRODUCT_ID, s.STORE_ID
            FROM store_table s
            INNER JOIN
                (SELECT PRODUCT_ID, min(DATE(DATE)), STORE_ID AS first_purchase
            FROM store_table
            WHERE STRFTIME('%Y', DATE(DATE)) = 2019);"""

### Question 3

In [ ]:
query3 = """SELECT
                STORE_ID,
                count(*) as total_orders
            FROM
                store_table
            WHERE
                DATE(DATE) between '2019-12-01' and '2019-12-31'
            GROUP BY 1;"""

This query may differ from the stores records because the same order number appears multiple times per store.
This would cause an over count in the number of orders because each order is counted more than once.

### Question 4
This contains a hacky conversion from month number to month name because SQLite doesn't support MONTHNAME (which would be
preferred over the (CASE strftime...) statement.

In [ ]:
query4 = """SELECT DISTINCT s.ORDER_ID,
                CASE
                    WHEN EXISTS (SELECT s2.ORDER_ID, s2.DATE FROM store_table s2
                                    WHERE
                                  (CASE strftime('%m', date(s2.DATE)) WHEN '01' THEN 'January' WHEN '02' THEN 'February' WHEN '03' THEN 'March' WHEN '04' THEN 'April' WHEN '05' THEN 'May' WHEN '06' THEN 'June' WHEN '07' THEN 'July' WHEN '08' THEN 'August' WHEN '09' THEN 'September' WHEN '10' THEN 'October' WHEN '11' THEN 'November' WHEN '12' THEN 'December' ELSE ''
                                        end) LIKE '%r')
                                THEN 'Y'
                                ELSE 'N'
                END month_ending_in_r,
                CASE
                    WHEN EXISTS (SELECT s3.ORDER_ID, s3.PRODUCT_ID, s3.QUANTITY
                                    FROM store_table s3
                                    WHERE s.ORDER_ID = s3.ORDER_ID
                                    GROUP BY s3.ORDER_ID, s3.PRODUCT_ID
                                    HAVING QUANTITY > 1)
                        THEN 'Y'
                        ELSE 'N'
                END contains_product_qty_greater_than_1
                FROM store_table s;"""

### Question 5

I ran out of time working on this question and was unable to verify the results.

In [ ]:
query5 = """SELECT count(DISTINCT USER_ID)
                FROM (SELECT s.USER_ID, s.ORDER_ID, date(s.DATE) AS DATEONLY, s.BRAND_ID, count(ORDER_ID)
                    FROM store_table s
                LEFT JOIN (SELECT b.BRAND_ID, b.HOLDING_COMPANY_NAME FROM brand_table b) as hc
                ON s.BRAND_ID = hc.BRAND_ID
                WHERE HOLDING_COMPANY_NAME = 'Diageo'
                GROUP BY
                        USER_ID,
                        ORDER_ID
                HAVING count(DISTINCT ORDER_ID) >= 3)
                WHERE DATEONLY >= '2020-07-15'"""

### Question 6

This regex will match only numbers between pipe delmiters.  It will ignore the starting header ("stores-") as long as
the header contains a "-" at the end.
This was tested with global search on.
```(?<=\-)(\d*?)(?=\|)|(?<=\|)(\d*?)(?=\|)|(?<=\|)(\d*?)(?=$|\s|\D)```


